-----

**MODULES USED**

-----------

In [ ]:
import requests
from lxml import etree
from selenium import webdriver
from time import sleep
import pandas as pd
from textblob import TextBlob as tb
import time
from bs4 import BeautifulSoup
import re, datetime

----

**FUNCTION TO SEARCH MOT EXPIRY DATE**

------

In [ ]:
def mot(s):               ####SOME MOT EXPIRY DATES WERE IN DESCRIPTION AND IN OTHER TEXT### "a" is the sentence containg MOT expiry
    try:
        
        if re.search('\d{2}-\d{2}-\d{4}', s):
            match = re.search('\d{2}-\d{2}-\d{4}', s)
            return datetime.datetime.strptime(match.group(), '%d-%m-%Y').date()
        
        if re.search('\d{2}-\d{4}', s):
            match = re.search('\d{2}-\d{4}', s)
            return datetime.datetime.strptime(match.group(), '%m-%Y').date()      

        else:
            blob = tb(s[0])
            months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
            mot = ""
            for i in range(len(blob.noun_phrases)):
                if blob.noun_phrases[i] == "mot":
                    for word in blob.noun_phrases[i:]:
                        mot = mot + " " + word
                        if word.lower() in months:
                            return mot
    except:
        return "N/A"

-----

**FIRST FUNCTION THAT SCRAPES THE PAGES AND RETURNs A LIST WITH ALL THE DATA IN IT**

SELENIUM OPENS THE THE VEHICLES PAGE AND THEN SETS THE FILTER TO MOST RECENTLY ADDED VEHICLES SO THAT IT'LL BE EASIER TO CHECK THE UPDATED VEHICLES AFTERWARDS.

------

In [ ]:
main = []              ###A GLOBAL LIST CONTAINING ALL THE SCRAPED DATA###

def run_first():           ###FUNCTION THAT SCRAPES THE GIVEN NUMBER OF PAGES###
    
    driver = webdriver.Chrome('/Users/vaibhav/Downloads/chromedriver')
    driver.get('https://www.usedcarsni.com')
    sleep(1)

    searchbox = driver.find_element_by_xpath('//button[contains(@class, "submit-search btn btn-warning btn-lg col-xss-12 col-md-4 col-md-push-3 col-lg-4 col-lg-push-4")]')
    searchbox.click()
    sleep(1)

    button = driver.find_element_by_xpath('/html/body/main/div[4]/nav[1]/div[3]/div/div[1]/span')
    button.click()

    sleep(1)

    button = driver.find_element_by_xpath('/html/body/main/div[4]/nav[1]/div[3]/div/div[2]/ul/li[4]/label')
    button.click()
    
    sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    parser = etree.HTMLParser()
    tree = etree.fromstring(str(soup), parser)
    
    s = "".join(tree.xpath('//*[contains(@class, "page-control-label")]/text()')).split('\n')
    s = "".join(s)
    number_of_pages = (int(int(s.strip().split(" ")[-1])/20)) + 1
    
    pages = input("ENTER # OF PAGES TO SCRAPE OR TYPE 'ALL' FOR SCRAPING ALL THE PAGES: ")
    
    if pages == 'ALL':
        pages = number_of_pages

    for i in range(int(pages)):
        print("Page #" + str(i) + " out of " + str(number_of_pages) + " completed")
        try:
            soup = BeautifulSoup(driver.page_source, 'lxml')
            parser = etree.HTMLParser()
            tree = etree.fromstring(str(soup), parser)

            for links in tree.xpath('//a[contains(text(), "More Details")]/@href'):
                temp = []
                
                temp.append('https://www.usedcarsni.com/' + links)
                
                r = requests.get('https://www.usedcarsni.com/' + links)
                tree = etree.fromstring(r.text, parser)
    
            ####NAME OF THE VEHICLE
                
                temp.append("".join(tree.xpath('//*[contains(@class, "car-name-link")]/text()')).strip())   
    
                
            ####COLLECTS VALUE OF THE HEADINGS IN THE LIST "l"
    
                l = ["Mileage", 'Location', 'Colour', 'Engine Size', 'Fuel Type', 'Transmission', 'Doors', 'Body Style', 'CO2 Emission']

                for i in l:
                    x = '//*[contains(@class, "technical-headers") and contains(text(),"'
                    w =  i
                    y = '")]/following-sibling::div/text()'
                    s = x+w+y
                    temp.append("".join(tree.xpath(s)))          
        
                
            ####Collects MOT date
                
                if tree.xpath('//*[contains(@class, "technical-headers") and contains(text(),"MOT Expiry")]/following-sibling::div/text()'):
                    temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(),"MOT Expiry")]/following-sibling::div/text()')))
        
                else:
                    a = tree.xpath('//*[contains(text(), "MOT")]/text()')
                    MOT = mot(a)
                    temp.append(MOT)
                
            ####TAX COST VALUE
                
                temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Tax Cost")]/following-sibling::div/a/text()')))
                
            ####INSURANCE
                
                temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Insurance")]/following-sibling::div/a[1]/text()')))
                
            ####SELLER TYPE(DEALER OR PRIVATE)
                
                if tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Seller")]/following-sibling::div/span/text()'):
                    temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Seller")]/following-sibling::div/span/text()')))    
                else:
                    temp.append('Dealer')
            
            #####DESCRIPTION
            
                temp.append("".join(tree.xpath('//*[contains(@class, "technical-info features-text")]/text()')[0]))
                
            ####CAR PRICE
                
                temp.append("".join(tree.xpath('//*[contains(@class, "row technical-textarea car-price-box")]/div[2]/div/span/text()')))
    
                main.append(temp)
    
            
        ####CLICKS ON THE NEXT PAGE
    
            button = driver.find_element_by_xpath('//li[contains(@class, "active")]/following-sibling::li[1]/a')
            button.click()
            sleep(1)
        except:
            pass
    driver.close()
    return main

-----

In [ ]:
def open_first_page():
    driver = webdriver.Chrome('/Users/vaibhav/Downloads/chromedriver')
    driver.get('https://www.usedcarsni.com')
    sleep(1)

    searchbox = driver.find_element_by_xpath('//button[contains(@class, "submit-search btn btn-warning btn-lg col-xss-12 col-md-4 col-md-push-3 col-lg-4 col-lg-push-4")]')
    searchbox.click()
    sleep(1)

    button = driver.find_element_by_xpath('/html/body/main/div[4]/nav[1]/div[3]/div/div[1]/span')
    button.click()

    sleep(1)

    button = driver.find_element_by_xpath('/html/body/main/div[4]/nav[1]/div[3]/div/div[2]/ul/li[4]/label')
    button.click()
    
    return driver

------

**REMOVING UNWANTED "\n" FROM OUR DATA, UPDATE CHECK FUNCTION AND SECOND FUNCTION**

------

In [ ]:
def remove_n(column):
    col = []
    for value in df[column]:
        col.append("".join(value.split('\n')))
    df[column] = col
    
    
def update(details, data):               
    temp = []
    
    temp.append('https://www.usedcarsni.com/' + details)
                
    r = requests.get('https://www.usedcarsni.com/' + details)
    parser = etree.HTMLParser()
    tree = etree.fromstring(r.text, parser)
    
    temp.append("".join(tree.xpath('//*[contains(@class, "car-name-link")]/text()')))
    
    l = ["Mileage", 'Location', 'Colour', 'Engine Size', 'Fuel Type', 'Transmission', 'Doors', 'Body Style', 'CO2 Emission']

    for i in l:
        x = '//*[contains(@class, "technical-headers") and contains(text(),"'
        w =  i
        y = '")]/following-sibling::div/text()'
        s = x+w+y
        temp.append("".join(tree.xpath(s)))
        
    
    if tree.xpath('//*[contains(@class, "technical-headers") and contains(text(),"MOT Expiry")]/following-sibling::div/text()'):
        temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(),"MOT Expiry")]/following-sibling::div/text()')))
        
    else:
        a = tree.xpath('//*[contains(text(), "MOT")]/text()')
        MOT = mot(a)
        temp.append(MOT)
        
    temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Tax Cost")]/following-sibling::div/a/text()')))
    temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Insurance")]/following-sibling::div/a[1]/text()')))
    if tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Seller")]/following-sibling::div/span/text()'):
        temp.append("".join(tree.xpath('//*[contains(@class, "technical-headers") and contains(text(), "Seller")]/following-sibling::div/span/text()')))    
    else:
        temp.append('Dealer')
            
    temp.append("".join(tree.xpath('//*[contains(@class, "technical-info features-text")]/text()')[0]))
    temp.append("".join(tree.xpath('//*[contains(@class, "row technical-textarea car-price-box")]/div[2]/div/span/text()')))
    
    data = data.insert(0, temp)
    temp = []
    return data




def update_check(driver, df, data):
    curr_page = driver.page_source
    soup = BeautifulSoup(curr_page, 'lxml')
    parser = etree.HTMLParser()
    tree = etree.fromstring(str(soup), parser)
    details = tree.xpath('//a[contains(text(), "More Details")]/@href')
    for i in range(1,21):
        x = "".join(tree.xpath('/html/body/main/div[4]/div[4]/div/article[' + str(i) + ']/div[3]/div[1]/a/p/strong/span/text()'))
        y = "".join(tree.xpath('/html/body/main/div[4]/div[4]/div/article[' + str(i) + ']/div[3]/div[1]/a/p/strong/text()'))
        x = x.split('\n')
        y = y.split('\n')
        x = "".join(x)
        y = "".join(y)
        z = x.strip() +" "+ y.strip()
        
        if "".join(df['Vehicle Name'][0].split(" ")) == "".join(z.split(" ")):
            return data
            
        elif "".join(df['Vehicle Name'][0].split(" ")) != "".join(z.split(" ")):
            print("DataFrame: "+ df['Vehicle Name'][0])
            print("Checking with: " + z)
            update(details[i-1], data)
        
        
    button = driver.find_element_by_xpath('//li[contains(@class, "active")]/following-sibling::li[1]/a')
    button.click()
    return update_check(driver, df, data)

def run_second(time_to_run_update, iterations, df, data):

    wakeup = time.time()
    i=0
    while i<iterations:
        wakeup += time_to_run_update
        print('start')
        
        driver = open_first_page()
        
        u = update_check(driver, df, data)
 
        while time.time()<wakeup:
            time.sleep(1)
    
        i = i+1
        driver.close()
    return u

-----

**RUN THE NEXT CELL TO START SCRAPING THE CURRENTLY AVAILABLE VEHICLES ON THE SITE**

-------

In [ ]:
data = run_first()        ###WILL SCRAPE ALL THE CURRENTLY AVAILABLE PAGES

----

**CREATING DATAFRAME**

------------

In [ ]:
columns = ["URL", "Vehicle Name", "Mileage", 'Location', 'Colour', 'Engine Size', 'Fuel Type', 'Transmission', 'Doors', 'Body Style', 'CO2 Emission', 'MOT Expiry', 'Tax Cost', 'Insurance', 'Seller', 'Description', 'Price']

df = pd.DataFrame(data, columns=columns)

remove_n('Insurance')
remove_n('Tax Cost')
remove_n('Description')
remove_n('Vehicle Name')
print(len(data))
df.head()

-----

**RUN THE NEXT CELL TO CHECK IF ANY NEW VEHICLE HAS BEEN ADDED TO THE LIST**

-------

In [ ]:
t = input("Time to check for updates again: ")
iterations = input("No. of times you want to check for updates: ")
u = run_second(int(t), int(iterations), df, data)      ###WILL CHECK FOR UPDATES AFTER EVERY ""time"" SECONDS for ""iterations"" times

-----

In [ ]:
columns = ["URL", "Vehicle Name", "Mileage", 'Location', 'Colour', 'Engine Size', 'Fuel Type', 'Transmission', 'Doors', 'Body Style', 'CO2 Emission', 'MOT Expiry', 'Tax Cost', 'Insurance', 'Seller', 'Description', 'Price']

df = pd.DataFrame(data, columns=columns)

remove_n('Insurance')
remove_n('Tax Cost')
remove_n('Description')
remove_n('Vehicle Name')
print(len(data))
df.head()

**ENTER THE PATH WHERE YOU WANT TO STORE THE COMPLETE SCRAPED CSV FILE**

------


In [ ]:
path = input("Enter path you want to store your file: ")
df.to_csv(path)        